In [129]:
import matplotlib.pyplot as plt
import numpy as np
import torch

np.set_printoptions(suppress=True)

from anchors import SCALES, RATIOS

%matplotlib inline

In [259]:
def generate_anchors(feature_map, scales, ratios):
    '''
        Apply scales along with ratio to generate anchor x1, y1, x2, y2
    '''
    feature_map
    batch_size, channels, grid_height, grid_width = feature_map.shape
    half_grid_height = grid_height / 2
    half_grid_width = grid_width / 2
    num_ratios = ratios.shape[0]
    num_scales = scales.shape[0]
    num_anchors_per_pxl = num_scales + num_ratios - 1
    
    # Note: Generate the offsets in widths and heights, based on aspect ratios
    # Note: vary the scales and keep the ratio fixed
    varied_scales_w = [ scales[idx] * np.sqrt(ratios[0]) for idx in range(num_ratios) ]
    # Note: vary the ratios, besides the used one, and keep the scales fixed
    varied_ratios_w = [ scales[0] / np.sqrt(ratios[idx]) for idx in range(1, num_ratios) ]
    w_offsets = np.concatenate([varied_scales_w, varied_ratios_w]) * grid_height / grid_width
    
    varied_scales_h = [ 0.5 * scales[idx] / np.sqrt(ratios[0]) for idx in range(num_ratios) ]
    varied_ratios_h = [ 0.5 * scales[0] / np.sqrt(ratios[idx]) for idx in range(1, num_ratios) ]
    h_offsets = np.concatenate([varied_scales_h, varied_ratios_h])
    
    # Note: Generate the grid cell centers and flatten the grids
    x_step = 1.0 / grid_width
    y_step = 1.0 / grid_height
    center_x = x_step * (np.arange(0, grid_width) + 0.5)
    center_y = y_step * (np.arange(0, grid_height) + 0.5)
    x_grid, y_grid = np.meshgrid(center_x, center_y, indexing='ij')
    x_grid_flat, y_grid_flat = x_grid.flatten(), y_grid.flatten()
    
    # Note: Per grid cell, add the anchor boxes coordinates (cx +- w_at_scale and cy +- h_at_scale)
    # Note: [x1i y1i x2i y2i, ..., ] -> repeat w*h times
    stacked_offsets = np.vstack((-w_offsets, -h_offsets, w_offsets, h_offsets)).T
    dims_offsets = np.repeat(stacked_offsets, grid_height * grid_width, axis=0)
    print(dims_offsets.shape)
    
    # Note: stack the offsets with the cx and cy pairs to create the tuple of x1, y1, x2, y2
    dims_centers = np.vstack((x_grid_flat, y_grid_flat, x_grid_flat, y_grid_flat)).T
    dims_centers = np.repeat(dims_centers, num_anchors_per_pxl, axis=0)
    
    return np.expand_dims(dims_centers + dims_offsets, axis=0)

In [260]:
feature_map = np.zeros((1, 3, 561, 728))
scales = np.array([0.75, 0.5, 0.25])
ratios = np.array([1, 2, 0.5])
anchors = generate_anchors(feature_map, scales, ratios)
print(anchors.shape)

(2042040, 4)
(1, 2042040, 4)


In [ ]:
def show_anchors():

In [214]:
a = np.array([
    [1, 2],
    [3, 4]
]).T
b = a.repeat(3, axis=0)
print(b)

[[1 3]
 [1 3]
 [1 3]
 [2 4]
 [2 4]
 [2 4]]


In [215]:
a = torch.tensor([
    [1, 2],
    [3, 4]
]).T
b = a.repeat(3, 1)
print(b.numpy())

[[1 3]
 [2 4]
 [1 3]
 [2 4]
 [1 3]
 [2 4]]
